# Build custom container for Vertex training

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/04-perarm-features-bandit


## Load env config

* use the prefix from `00-env-setup`

In [2]:
PREFIX = 'mabv1'

In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "wortz-project-352116"
PROJECT_NUM              = "679926387543"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "679926387543-compute@developer.gserviceaccount.com"

PREFIX                   = "mabv1"
VERSION                  = "v1"

BUCKET_NAME              = "mabv1-wortz-project-352116-bucket"
BUCKET_URI               = "gs://mabv1-wortz-project-352116-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://mabv1-wortz-project-352116-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

VPC_NETWORK_FULL         = "projects/679926387543/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_ID      = "wortz-project-352116.movielens_dataset_mabv1"
BIGQUERY_TABLE_ID        = "wortz-project-352116.movielens_dataset_mabv1.training_dataset"

REPO_D

In [4]:
# ! gsutil ls $BUCKET_URI

gs://mabv1-wortz-project-352116-bucket/config/
gs://mabv1-wortz-project-352116-bucket/data/
gs://mabv1-wortz-project-352116-bucket/generative-trajectories/
gs://mabv1-wortz-project-352116-bucket/neural-linear-bandits-v1/
gs://mabv1-wortz-project-352116-bucket/vocabs/


## imports

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Build Image

## Container Image Variables

In [6]:
DOCKERNAME                = f'Dockerfile_perarm_feats'

REPOSITORY                = f'rl-movielens-{PREFIX}'
IMAGE_NAME                = f'train-perarm-feats-{VERSION}'

REMOTE_IMAGE_NAME         = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE_NAME}"

print(f"DOCKERNAME        = {DOCKERNAME}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME        = {IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")

DOCKERNAME        = Dockerfile_perarm_feats
REPOSITORY        = rl-movielens-mabv1
IMAGE_NAME        = train-perarm-feats-v1
REMOTE_IMAGE_NAME = us-central1-docker.pkg.dev/wortz-project-352116/rl-movielens-mabv1/train-perarm-feats-v1


## Create Artifact Repository

If you don't have an existing artifact repository, create one using the gcloud command below

In [7]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

## Local Docker build

Provide a name for your dockerfile and make sure you are authenticated

In [8]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


### Create Dockerfile

In [9]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

In [65]:
%%writefile Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cu113.py310

#updated to be a derivative image - good for general GCP usage

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

# COPY requirements.txt $APP_HOME/requirements.txt
COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

# ADD src $APP_HOME/src
# RUN ls src
COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

RUN ls $APP_HOME

RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/

# Sets up the entry point to invoke the task.
# ENTRYPOINT ["python3", "-m", "src.FM_reward_trajectory_to_gcs.04a-build-FM-reward-trajectories-to-gcs"]

Overwriting Dockerfile


### Build Image Locally

In [66]:
!docker build -t $REMOTE_IMAGE_NAME -f Dockerfile .

Sending build context to Docker daemon  117.9MB
Step 1/12 : FROM gcr.io/deeplearning-platform-release/base-cu113.py310
 ---> 205814b695ff
Step 2/12 : ENV PYTHONUNBUFFERED True
 ---> Using cache
 ---> 7f6014521740
Step 3/12 : ENV APP_HOME /workspace
 ---> Using cache
 ---> 535abfc7c18d
Step 4/12 : WORKDIR $APP_HOME
 ---> Using cache
 ---> b9ef44c23ddb
Step 5/12 : COPY /requirements.txt $APP_HOME/requirements.txt
 ---> Using cache
 ---> 62c7f342c087
Step 6/12 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 97ea113bf6f6
Step 7/12 : RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt
 ---> Using cache
 ---> eaecf388790e
Step 8/12 : RUN ls $APP_HOME
 ---> Using cache
 ---> e42508e19c1d
Step 9/12 : COPY src/perarm_features $APP_HOME/src/perarm_features
 ---> Using cache
 ---> aabc05e3a99a
Step 10/12 : COPY src/per_arm_rl $APP_HOME/src/per_arm_rl
 ---> Using cache
 ---> f80057c3c7df
Step 11/12 : RUN ls $APP_HOME
 ---> Using cache
 ---> 346d41f4d45f
Step 12/12 : RUN export PYT

In [67]:
# !gcloud builds submit ./ --region $REGION --tag $REMOTE_IMAGE_NAME 

### Push container to Registry

In [68]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/wortz-project-352116/rl-movielens-mabv1/train-perarm-feats-v1]

aa9b6409: Preparing 
8273c905: Preparing 
7fc69c53: Preparing 
425b7f44: Preparing 
5d07c296: Preparing 
9cf5a610: Preparing 
44331350: Preparing 
94b6f2e1: Preparing 
c583dd12: Preparing 
68fa50ca: Preparing 
104fe16a: Preparing 
d0a58cfb: Preparing 
1af02baf: Preparing 
ec1e6678: Preparing 
9095b2b5: Preparing 
3aa9f0f9: Preparing 
099988dd: Preparing 
77c1d361: Preparing 
87f685e3: Preparing 
09c02995: Preparing 
c7b52419: Preparing 
0a840444: Preparing 
309fab1f: Preparing 
0e35d7c6: Preparing 
70adb775: Preparing 
cebba87f: Preparing 
bf18a086: Preparing 
9e367d95: Preparing 
9e8927e0: Preparing 
ad0ce3ec: Preparing 
17453a10: Preparing 
e1a1cbfb: Preparing 
7090d6d6: Preparing 
1fa7f0e3: Preparing 
ef4e19ae: Preparing 
d9356ee8: Preparing 
66373eb9: Preparing 
7a859620: Preparing 
183bef21: Preparing 
69562af2: Preparing 
d8cea54a: La

In [91]:
VERTEX_SA

'679926387543-compute@developer.gserviceaccount.com'

In [95]:
from google.cloud import aiplatform


experiment = "generative-trajectories-mab"
aiplatform.init(project=PROJECT_ID, location=LOCATION, experiment=experiment)

display_name = "generative-trajectories-mab"
script_path = "04-perarm-features-bandit/04a-build-FM-reward-trajectories-to-gcs.py"

job = aiplatform.CustomJob.from_local_script(
    display_name=display_name,
    script_path=script_path,
    container_uri=REMOTE_IMAGE_NAME,
    enable_autolog=True,
    staging_bucket=BUCKET_URI
)

job.run(service_account=VERTEX_SA,
        experiment=experiment,
        sync=False
)

Training script copied to:
gs://mabv1-wortz-project-352116-bucket/aiplatform-2023-08-22-19:46:58.315-aiplatform_custom_trainer_script-0.1.tar.gz.
Associating projects/679926387543/locations/us-central1/metadataStores/default/contexts/generative-trajectories-mab-generative-trajectories-mab-80af9 to Experiment: generative-trajectories-mab
Creating CustomJob
CustomJob created. Resource name: projects/679926387543/locations/us-central1/customJobs/7998570578057363456
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/679926387543/locations/us-central1/customJobs/7998570578057363456')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7998570578057363456?project=679926387543
CustomJob projects/679926387543/locations/us-central1/customJobs/7998570578057363456 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/679926387543/locations/us-central1/customJobs/7998570578057363456 current state:
JobState.JOB_S